In [1]:
import sys
sys.path.append('/Users/vishwajit/.pyenv/versions/3.7.3/lib/python3.7/site-packages')
import pandas as pd
from census import Census
from us import states
from bs4 import BeautifulSoup
import csv
import requests
import numpy as np
import time

In [82]:
url = 'https://api.census.gov/data/2018/acs/acs5/profile/variables.html'
html = requests.get(url)
soup = BeautifulSoup(html.content,'html.parser')
table = soup.find("table")

In [83]:
output_rows = []
for table_row in table.findAll('tr'):
    columns = table_row.findAll('td')
    output_row = []
    for column in columns:
        output_row.append(column.text)
    output_rows.append(output_row)
    
with open('Attributes2.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(output_rows)

In [2]:
attributes = pd.read_csv('Attributes2.csv',index_col=None)
attributes.reset_index(inplace=True)

In [3]:
attributes.drop(['level_2','level_3','level_4','level_5','level_6','1384 variables'],axis=1,inplace=True)
attributes.rename(columns={'level_0':'Name','level_1':'Label'},inplace=True)

In [94]:
attributes

,Name,Label
0,AIANHH,Geography
1,AIHHTL,Geography
2,AIRES,Geography
3,ANRC,Geography
4,CBSA,Geography
...,...,...
1379,TRIBALSUB,Geography
1380,TRISUBREM,Geography
1381,UA,Geography
1382,ucgid,Uniform Census Geography Identifier clause


In [116]:
data_profile_attr = []
for i in range(len(attributes)):
    if(attributes['Name'][i][:3]=='DP0' and attributes['Name'][i][-2:]=='PE'):
        data_profile_attr.append(attributes['Name'][i])

In [79]:
key='e9bd708b19f93ec346fe324b7c3d3bb8cf5de08d'
for group in ['DP02','DP03','DP04','DP05','DP02PR']:
    start = time.time()
    print(group)
    url = 'https://api.census.gov/data/2018/acs/acs5/profile?key='+key+'&get=group('+group+')&for=zip%20code%20tabulation%20area:*'
    html = requests.get(url)
    soup = BeautifulSoup(html.content,'html.parser')
    s = soup.text.replace('\n','')
    s = s.replace('null','"null"')
    l = eval(s)
    headers = l.pop(0)
    temp_df = pd.DataFrame(l,columns=headers)
    temp_df.set_index('zip code tabulation area',inplace=True)
    temp_df.drop(['GEO_ID','NAME'],axis=1,inplace=True)
    if(group=='DP02'):
        df = temp_df
    else:
        df = df.join(temp_df)
    print(time.time()-start)

DP05
996.1548550128937
DP02PR
1507.0254759788513


In [80]:
df

,GEO_ID,DP02_0001E,DP02_0001M,DP02_0001PE,DP02_0001PM,DP02_0002E,DP02_0002M,DP02_0002PE,DP02_0002PM,DP02_0003E,...,DP02PR_0150PEA,DP02PR_0150PMA,DP02PR_0151MA,DP02PR_0151EA,DP02PR_0151PMA,DP02PR_0151PEA,DP02PR_0152EA,DP02PR_0152MA,DP02PR_0152PEA,DP02PR_0152PMA
zip code tabulation area,,,,,,,,,,,,,,,,,,,,,
43964,8600000US43964,3811,175,3811,-888888888,2383,163,62.5,4.0,893,...,null,null,null,null,null,null,null,null,null,null
28216,8600000US28216,19911,449,19911,-888888888,12010,495,60.3,2.4,5797,...,null,null,null,null,null,null,null,null,null,null
28277,8600000US28277,26695,385,26695,-888888888,19512,472,73.1,1.6,10393,...,null,null,null,null,null,null,null,null,null,null
28278,8600000US28278,9176,272,9176,-888888888,6985,297,76.1,2.8,3467,...,null,null,null,null,null,null,null,null,null,null
28303,8600000US28303,13207,327,13207,-888888888,6849,355,51.9,2.3,2738,...,null,null,null,null,null,null,null,null,null,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98279,8600000US98279,340,52,340,-888888888,190,35,55.9,7.1,40,...,null,null,null,null,null,null,null,null,null,null
98280,8600000US98280,238,62,238,-888888888,124,41,52.1,17.0,21,...,null,null,null,null,null,null,null,null,null,null
98311,8600000US98311,10146,321,10146,-888888888,7280,355,71.8,3.1,2964,...,null,null,null,null,null,null,null,null,null,null


In [81]:
df.to_csv('Data_Profiles.csv')

In [117]:
df1 = df[data_profile_attr]

In [118]:
df1

,DP02_0001E,DP02_0002E,DP02_0003E,DP02_0004E,DP02_0005E,DP02_0006E,DP02_0007E,DP02_0008E,DP02_0009E,DP02_0010E,...,DP05_0080E,DP05_0081E,DP05_0082E,DP05_0083E,DP05_0084E,DP05_0085E,DP05_0086E,DP05_0087E,DP05_0088E,DP05_0089E
zip code tabulation area,,,,,,,,,,,,,,,,,,,,,
43964,3811,2383,893,1795,588,234,81,354,224,1428,...,21,0,0,127,0,127,4155,6811,3299,3512
28216,19911,12010,5797,6754,2757,1121,626,4135,2414,7901,...,1032,0,144,1177,27,1150,21688,36519,16183,20336
28277,26695,19512,10393,16546,8691,580,281,2386,1421,7183,...,10486,0,231,2092,107,1985,27830,46233,21377,24856
28278,9176,6985,3467,5544,2756,483,285,958,426,2191,...,1709,53,55,1627,9,1618,9720,18464,8545,9919
28303,13207,6849,2738,4511,1527,425,204,1913,1007,6358,...,1080,132,55,1491,10,1481,15525,22326,10714,11612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98279,340,190,40,174,28,14,12,2,0,150,...,4,0,0,11,0,11,697,567,308,259
98280,238,124,21,124,21,0,0,0,0,114,...,20,0,0,3,0,3,489,357,167,190
98311,10146,7280,2964,5549,2064,573,298,1158,602,2866,...,2541,403,20,2095,17,2078,10856,20729,10667,10062


In [119]:
df1.to_csv('Data_Profiles1.csv')

In [121]:
attributes

,Label
Name,
AIANHH,Geography
AIHHTL,Geography
AIRES,Geography
ANRC,Geography
CBSA,Geography
...,...
TRIBALSUB,Geography
TRISUBREM,Geography
UA,Geography


In [130]:
for att in data_profile_attr:
    print(attributes[attributes['Name']==att].values)

[['DP02_0001E' 'Estimate!!HOUSEHOLDS BY TYPE!!Total households']]
[['DP02_0002E'
  'Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)']]
[['DP02_0003E'
  'Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!With own children of the householder under 18 years']]
[['DP02_0004E'
  'Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Married-couple family']]
[['DP02_0005E'
  'Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Married-couple family!!With own children of the householder under 18 years']]
[['DP02_0006E'
  'Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Male householder, no wife present, family']]
[['DP02_0007E'
  'Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Family households (families)!!Male householder, no wife present, family!!With own children of the householder under 18 years']]
[['DP02_0008E'
  'Estimate!!HOUSEHOLDS BY TYPE!!Total h

  'Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree']]
[['DP02PR_0066E'
  'Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher']]
[['DP02PR_0067E'
  "Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher"]]
[['DP02PR_0068E'
  'Estimate!!VETERAN STATUS!!Civilian population 18 years and over']]
[['DP02PR_0069E'
  'Estimate!!VETERAN STATUS!!Civilian population 18 years and over!!Civilian veterans']]
[['DP02PR_0070E'
  'Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!Total Civilian Noninstitutionalized Population']]
[['DP02PR_0071E'
  'Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!Total Civilian Noninstitutionalized Population!!With a disability']]
[['DP02PR_0072E'
  'Estimate!!DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION!!Under 18 years']]
[['DP02PR_0073E'
  'Estimate!!DISABILITY STA

  'Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!Married couple families!!With related children of the householder under 18 years']]
[['DP03_0124E'
  'Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!Married couple families!!With related children of the householder under 18 years!!With related children of the householder under 5 years only']]
[['DP03_0125E'
  'Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!Families with female householder, no husband present']]
[['DP03_0126E'
  'Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!Families with female householder, no husband present!!With related children of the householder under 18 years']]
[['DP03_0127E'
  'Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY

[['DP05_0058E' 'Estimate!!RACE!!Total population!!Two or more races']]
[['DP05_0059E'
  'Estimate!!RACE!!Total population!!Two or more races!!White and Black or African American']]
[['DP05_0060E'
  'Estimate!!RACE!!Total population!!Two or more races!!White and American Indian and Alaska Native']]
[['DP05_0061E'
  'Estimate!!RACE!!Total population!!Two or more races!!White and Asian']]
[['DP05_0062E'
  'Estimate!!RACE!!Total population!!Two or more races!!Black or African American and American Indian and Alaska Native']]
[['DP05_0063E'
  'Estimate!!Race alone or in combination with one or more other races!!Total population']]
[['DP05_0064E'
  'Estimate!!Race alone or in combination with one or more other races!!Total population!!White']]
[['DP05_0065E'
  'Estimate!!Race alone or in combination with one or more other races!!Total population!!Black or African American']]
[['DP05_0066E'
  'Estimate!!Race alone or in combination with one or more other races!!Total population!!American Indi